In [3]:
import pandas as pd
import numpy as np

file_path = r"C:\Users\Kanchan\Downloads\cancer_industry_pollution_dataset.xlsx"

patients = pd.read_excel(file_path, sheet_name="Patients")
industries = pd.read_excel(file_path, sheet_name="Industries")
pollutants = pd.read_excel(file_path, sheet_name="Pollutants")

patients.head(), industries.head(), pollutants.head()


(   Patient_ID       Name  Age Sex      Village District Cancer_Type  \
 0           1  Patient_1   82   M    Village_7   Nagpur       Liver   
 1           2  Patient_2   18   M  Village_174   Mumbai      Throat   
 2           3  Patient_3    5   M   Village_24    Thane       Liver   
 3           4  Patient_4   72   M  Village_184  Solapur       Liver   
 4           5  Patient_5   76   F    Village_2   Raigad      Breast   
 
    Diagnosis_Year  
 0            2017  
 1            2023  
 2            2010  
 3            2024  
 4            2020  ,
    Industry_ID Industry_Name  Industry_Type      Village District   Latitude  \
 0            1    Industry_1        Textile  Village_152   Raigad  20.199462   
 1            2    Industry_2         Pharma   Village_52   Nashik  17.405515   
 2            3    Industry_3  Petrochemical   Village_80   Raigad  17.013746   
 3            4    Industry_4        Plastic    Village_7  Solapur  18.019330   
 4            5    Industry_5     

In [5]:
print("Patients shape:", patients.shape)
print("Industries shape:", industries.shape)
print("Pollutants shape:", pollutants.shape)

print("\nMissing values in each dataset:")
print("Patients:", patients.isnull().sum().sum())
print("Industries:", industries.isnull().sum().sum())
print("Pollutants:", pollutants.isnull().sum().sum())


Patients shape: (50000, 8)
Industries shape: (5000, 8)
Pollutants shape: (45000, 6)

Missing values in each dataset:
Patients: 0
Industries: 0
Pollutants: 0


In [7]:
def clean_text(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.upper()
    return df

patients = clean_text(patients)
industries = clean_text(industries)
pollutants = clean_text(pollutants)

patients.head()

,Patient_ID,Name,Age,Sex,Village,District,Cancer_Type,Diagnosis_Year
0,1,PATIENT_1,82,M,VILLAGE_7,NAGPUR,LIVER,2017
1,2,PATIENT_2,18,M,VILLAGE_174,MUMBAI,THROAT,2023
2,3,PATIENT_3,5,M,VILLAGE_24,THANE,LIVER,2010
3,4,PATIENT_4,72,M,VILLAGE_184,SOLAPUR,LIVER,2024
4,5,PATIENT_5,76,F,VILLAGE_2,RAIGAD,BREAST,2020


In [12]:
from tqdm import tqdm
import numpy as np
import pandas as pd

# Prepare a list to store results
results = []

# Process district-by-district to avoid memory crash
for district in tqdm(patients['District'].unique(), desc="Processing districts"):
    
    # Filter data for this district
    p = patients[patients["District"] == district].copy()
    i = industries[industries["District"] == district].copy()
    
    # If no industries in that district, skip
    if i.empty:
        continue
    
    # Create cartesian merge safely
    p['key'] = 1
    i['key'] = 1
    merged = p.merge(i, on='key', suffixes=('_PATIENT', '_INDUSTRY')).drop('key', axis=1)
    
    # Distance calculation (approx)
    merged['Distance_km'] = (
        np.sqrt(
            (merged['Latitude_INDUSTRY'] - merged['Latitude_INDUSTRY'])**2 +
            (merged['Longitude_INDUSTRY'] - merged['Longitude_INDUSTRY'])**2
        ) * 111
    )
    
    # Take nearest industry only
    nearest = merged.loc[merged.groupby("Patient_ID")['Distance_km'].idxmin()]
    
    # Collect
    results.append(nearest)

# Combine all districts
patients_industries = pd.concat(results, ignore_index=True)

patients_industries.head()
print("🎉 Done — safe merge completed without memory overflow!")


Processing districts:   0%|                                                                      | 0/8 [00:13<?, ?it/s]


KeyError: 'Latitude_INDUSTRY'

In [13]:
# Rename industry longitude/latitude to avoid suffix issues
industries = industries.rename(columns={
    "Latitude": "Industry_Lat",
    "Longitude": "Industry_Long"
})


In [14]:
district_coordinates = {
    "PUNE": (18.5204, 73.8567),
    "MUMBAI": (19.0760, 72.8777),
    "RAIGAD": (18.5158, 73.1822),
    "THANE": (19.2183, 72.9781),
    "NAGPUR": (21.1458, 79.0882),
    "KOLHAPUR": (16.7050, 74.2433),
    "SOLAPUR": (17.6599, 75.9064),
    "NASHIK": (19.9975, 73.7898)
}

patients[['Patient_Lat','Patient_Long']] = patients['District'].map(
    district_coordinates
).apply(pd.Series)


In [15]:
from tqdm import tqdm
import numpy as np
import pandas as pd

results = []

for district in tqdm(patients["District"].unique(), desc="Processing districts"):

    p = patients[patients["District"] == district].copy()
    i = industries[industries["District"] == district].copy()

    if i.empty:
        continue

    p["key"] = 1
    i["key"] = 1
    merged = p.merge(i, on="key").drop("key", axis=1)

    # Correct distance calculation
    merged["Distance_km"] = (
        np.sqrt(
            (merged["Patient_Lat"] - merged["Industry_Lat"])**2 +
            (merged["Patient_Long"] - merged["Industry_Long"])**2
        ) * 111  # degrees → km conversion
    )

    nearest = merged.loc[merged.groupby("Patient_ID")["Distance_km"].idxmin()]
    results.append(nearest)

patients_industries = pd.concat(results, ignore_index=True)
patients_industries.head()


Processing districts: 100%|██████████████████████████████████████████████████████████████| 8/8 [00:17<00:00,  2.20s/it]


,Patient_ID,Name,Age,Sex,Village_x,District_x,Cancer_Type,Diagnosis_Year,Coordinates,Patient_Lat,Patient_Long,Industry_ID,Industry_Name,Industry_Type,Village_y,District_y,Industry_Lat,Industry_Long,MPCB_Category,Distance_km
0,1,PATIENT_1,82,M,VILLAGE_7,NAGPUR,LIVER,2017,"(21.1458, 79.0882)",21.1458,79.0882,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121
1,11,PATIENT_11,30,M,VILLAGE_98,NAGPUR,BREAST,2021,"(21.1458, 79.0882)",21.1458,79.0882,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121
2,23,PATIENT_23,83,F,VILLAGE_29,NAGPUR,BREAST,2015,"(21.1458, 79.0882)",21.1458,79.0882,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121
3,24,PATIENT_24,59,M,VILLAGE_185,NAGPUR,THROAT,2015,"(21.1458, 79.0882)",21.1458,79.0882,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121
4,25,PATIENT_25,65,M,VILLAGE_161,NAGPUR,THROAT,2016,"(21.1458, 79.0882)",21.1458,79.0882,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121


In [16]:
# Convert cancer risk to score
risk_map = {"HIGH":3, "MODERATE":2, "LOW":1}
pollutants['Risk_Score'] = pollutants["Cancer_Risk"].map(risk_map)

# Compute average risk per industry
industry_risk = pollutants.groupby("Industry_ID")["Risk_Score"].mean().reset_index()
industry_risk = industry_risk.rename(columns={"Risk_Score":"Avg_Pollutant_Risk"})

# Merge risk score into patient mapping
final_dataset = patients_industries.merge(industry_risk, on="Industry_ID", how="left")
final_dataset.head()


,Patient_ID,Name,Age,Sex,Village_x,District_x,Cancer_Type,Diagnosis_Year,Coordinates,Patient_Lat,...,Industry_ID,Industry_Name,Industry_Type,Village_y,District_y,Industry_Lat,Industry_Long,MPCB_Category,Distance_km,Avg_Pollutant_Risk
0,1,PATIENT_1,82,M,VILLAGE_7,NAGPUR,LIVER,2017,"(21.1458, 79.0882)",21.1458,...,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121,2.75
1,11,PATIENT_11,30,M,VILLAGE_98,NAGPUR,BREAST,2021,"(21.1458, 79.0882)",21.1458,...,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121,2.75
2,23,PATIENT_23,83,F,VILLAGE_29,NAGPUR,BREAST,2015,"(21.1458, 79.0882)",21.1458,...,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121,2.75
3,24,PATIENT_24,59,M,VILLAGE_185,NAGPUR,THROAT,2015,"(21.1458, 79.0882)",21.1458,...,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121,2.75
4,25,PATIENT_25,65,M,VILLAGE_161,NAGPUR,THROAT,2016,"(21.1458, 79.0882)",21.1458,...,4800,INDUSTRY_4800,AUTOMOBILE,VILLAGE_30,NAGPUR,21.236867,79.408432,RED,36.955121,2.75


In [17]:
final_dataset.to_csv("patients_industries_clean.csv", index=False)
industries.to_csv("industries_clean.csv", index=False)
pollutants.to_csv("pollutants_clean.csv", index=False)

print("✔️ Clean datasets saved successfully!")


✔️ Clean datasets saved successfully!


In [18]:
import os
print(os.getcwd())


C:\Users\Kanchan
